In [4]:
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

import numpy as np
import DataPreprocessing

In [2]:
combs2 = ['{},{}'.format(i,j) for i in range(1,5) for j in range(i,5) if i!=j]
combs3 = ['{},{},{}'.format(i,j,k) for i in range(1, 5) for j in range(i,5) for k in range(j,5) if i!=j and i!=k and j!=k]
combs = combs2 + combs3
combs = ['1,3','1,4','2,3','2,4','1,2,3','1,2,4','1,3,4','2,3,4','1,2,3','1,2,4','1,3,4','2,3,4']
combs = []

In [5]:
me_train, mom_train = DataPreprocessing.npy('LO/MG_uuuxux/me_4jet_1000000.npy', 'LO/MG_uuuxux//mom_4jet_1000000.npy', combs, 1000, frac=1)
me_test, mom_test = DataPreprocessing.npy('LO/MG_uuuxux/me_4jet_100000.npy', 'LO/MG_uuuxux/mom_4jet_100000.npy', combs, 1000, frac=1)

##Rescale
scaler = StandardScaler()

mom_train = scaler.fit_transform(mom_train) #Rescale on training set
mom_test = scaler.transform(mom_test) #Rescale on test set

In [6]:
# Random Model from Tensorflow Website
def build_model(dropout, hidden_layers=5, components=200, optimizer='RMSprop'):
    
  model = keras.Sequential([
    keras.layers.Dense(components, activation='relu', input_shape=[12]), #could try swish??
    keras.layers.BatchNormalization(),  
    keras.layers.Dropout(dropout),
      
    *np.repeat([keras.layers.Dense(components, activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(dropout)], hidden_layers),
    
    keras.layers.Dense(components, activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(dropout),
    
    keras.layers.Dense(1) 
  ])

  model.compile(loss='mse', #gans?
                optimizer=optimizer,
                metrics=['mse'])
  return model

In [7]:
model = KerasRegressor(build_fn=build_model, epochs=3)

dropout = [0.3,0.4,0.5]
param_grid = dict(dropout=dropout)

grid = RandomizedSearchCV(model, param_grid, cv=3)
grid_result = grid.fit(mom_train[:100000], me_train[:100000])

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Instructions for updating:
keep_dims is deprecated, use keepdims instead


/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:530: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: Error when checking input: expected dense_1_input to have shape (None, 12) but got array with shape (66666, 16)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [6]:
#model = build_model(grid_result.best_params_['dropout'])
model = build_model(0.5)
model.fit(mom_train, me_train, epochs=3)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/3
300000/300000 [==============================]300000/300000 [==============================] - 71s 236us/step - loss: 0.0501 - mean_squared_error: 0.0501

Epoch 2/3
300000/300000 [==============================]300000/300000 [==============================] - 71s 237us/step - loss: 1.4324e-04 - mean_squared_error: 1.4324e-04

Epoch 3/3
300000/300000 [==============================]300000/300000 [==============================] - 72s 239us/step - loss: 1.2855e-04 - mean_squared_error: 1.2855e-04



In [8]:
model.evaluate(mom_test, me_test, batch_size=64, verbose=1)

50000/50000 [==============================]50000/50000 [==============================] - 2s 37us/step



[6344488.30976, 6344488.30976]

In [9]:
print(model.metrics_names)

['loss', 'mean_squared_error']


In [13]:
print(np.mean(me_train))
print(np.std(me_train))

0.06921789967290297
0.024785264292307374


In [14]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(100*np.divide(np.abs((y_true - y_pred),y_true)))

In [ ]:
me_predict = model.predict(mom_test)

print(np.mean(100*np.divide(np.abs(me_test - me_predict), me_test)))

In [ ]:
print(model.predict(mom_test[:10]))
print(me_test[:10])